Synthetic Shapes akazeデータセットの作成方法
特徴点検出手法akazeを使用して、Symthetic Shapesデータセットの画像から特徴点を検出

In [1]:
import numpy as np
import glob
import cv2 as cv
import matplotlib.pyplot as plt
import os
from os.path import join
from natsort import natsorted
import shutil

DATA_PATH = '../datasets'  # 環境に合わせて変更する

In [2]:
def keypoint_coordinate_list(kp, a):
    kp_list = []
    for k in kp:
        # kp_list.append([k.pt[0], k.pt[1]]) # (w,h)
        kp_list.append([k.pt[1]/a, k.pt[0]/a]) # (h,w)
    return np.round(np.array(kp_list), decimals=3)

def detect_feature_points_with_resize(img, a):
    width = img.shape[1]
    height = img.shape[0]
    img = cv.resize(img, (width*a, height*a))
    # AKAZE検出器を読み込む
    akaze = cv.AKAZE_create()
    # 特徴点の検出
    kp = akaze.detect(img)
    kp_list = keypoint_coordinate_list(kp, a)
    return kp_list

In [7]:
# gaussian noiseは特徴点は出ないためここでは除く
draws = [
    "draw_checkerboard",
    "draw_cube",
    "draw_ellipses",
    "draw_lines",
    "draw_multiple_polygons",
    "draw_polygon",
    "draw_star",
    "draw_stripes",
    ]
datasets = ["training", "validation", "test"]
original_dataset = f'{DATA_PATH}/synthetic_shapes_v6'
new_dataset = f'{DATA_PATH}/synthetic_shapes_akaze'

######データセット構造の作成######
for d in draws:
    shutil.copytree(
        f'{original_dataset}/{d}/images',
        f'{new_dataset}/{d}/images')
    os.makedirs(f'{new_dataset}/{d}/points', exist_ok=True)
    for data in datasets:
        os.makedirs(f'{new_dataset}/{d}/points/{data}', exist_ok=True)

In [11]:
for d in draws:
    draw_path = join(new_dataset, d)
    for data in datasets:
        # 画像ファイルのパス
        img_paths = join(draw_path, "images", data, "*")
        # natsorted : ファイルを順番に並べる(0,1,2,...)
        img_paths = natsorted(glob.glob(img_paths))
        npy_paths = [i.replace("images", "points").replace("png", "npy") for i in img_paths]
        for img_path, npy_path in zip(img_paths, npy_paths):
            img = cv.imread(img_path)
            # akaze特徴点検出、出力は特徴点の座標リスト
            kp_list = detect_feature_points_with_resize(img, 2)
            if kp_list.size != 0: # 特徴点が検出されたかされなかったか
                # npyファイルに座標リストを保存
                np.save(npy_path, kp_list)
            else:
                # 特徴点がない場合は画像を消す
                os.remove(img_path)

In [14]:
all = 0
for d in draws:
    draw_path = join(new_dataset, d)
    for data in datasets:
        img_paths = join(draw_path, "images", data, "*")
        img_paths = natsorted(glob.glob(img_paths))
        all += len(img_paths)

print(f'総データ数：{all}')


総データ数：78493
